In [54]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import SGD
from torch.optim.lr_scheduler import LinearLR
import numpy as np
import pandas as pd
import cv2
import os

### GPU 사용 설정

In [97]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

### 데이터 가져오기

In [17]:
# /content/drive/MyDrive/고모부_머신러닝/dogncat
paths = []
dataset_type = []
labels = []

def make_dataframe(dirpath):
    for dirname, _, filenames in os.walk(dirpath):
        for filename in filenames:
            file_path = dirname+'/'+filename
            paths.append(file_path)

            if '/training_set' in file_path:
                dataset_type.append('train')
            elif '/test_set' in file_path:
                dataset_type.append('test')
            else:
                dataset_type.append('N/A')
            
            if 'dogs' in file_path:
                labels.append('DOG')
            elif 'cats' in file_path:
                labels.append('CAT')
            else:
                labels.append('N/A')

    df = pd.DataFrame({'path' : paths, 'type' : dataset_type, 'label' : labels})

    return df

In [18]:
cnd_df = make_dataframe('/content/drive/MyDrive/고모부_머신러닝/dogncat')
cnd_df.head()

,path,type,label
0,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
1,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
2,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
3,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT
4,/content/drive/MyDrive/고모부_머신러닝/dognc...,test,CAT


In [19]:
cnd_df = cnd_df[cnd_df['path'].str.contains('.jpg')].copy()
cnd_df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 10028 entries, 0 to 10031
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   path    10028 non-null  object
 1   type    10028 non-null  object
 2   label   10028 non-null  object
dtypes: object(3)
memory usage: 313.4+ KB


In [33]:
# train, valid dataset 만들기

from sklearn.model_selection import train_test_split

# train, validation 분리
train_df, valid_df = train_test_split(cnd_df[cnd_df['type']=='train'], test_size = 0.25, )

# train dataset
train_data = train_df['path'].values
train_label = train_df['label'].values
train_label_indices = train_df['label'].replace(['CAT', 'DOG'], [0, 1]).values

# validation dataset
valid_data = valid_df['path'].values
valid_label = valid_df['label'].values
valid_label_indices = valid_df['label'].replace(['CAT', 'DOG'], [0,1]).values


In [150]:
sample_tr = train_df.sample(10)
sample_val = valid_df.sample(10)

print(sample_tr.head(10))
print(sample_val.head(10))

                                                   path   type label
7618  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   DOG
9922  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   DOG
4922  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
2832  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
2393  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
4898  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
9760  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   DOG
2924  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
6997  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   DOG
7868  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   DOG
                                                   path   type label
3552  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
2372  /content/drive/MyDrive/고모부_머신러닝/dognc...  train   CAT
6647  /content/drive/MyDrive/고모ᄇ

In [165]:
sample_tr_data = sample_tr['path'].values
sample_tr_label = sample_tr['label'].replace(['CAT', 'DOG'], [0,1]).values
sample_val_data = sample_val['path'].values
sample_val_label = sample_val['label'].replace(['CAT', 'DOG'], [0,1]).values

In [166]:
print(sample_tr_data)
print(sample_tr_label)

['/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.3932.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.1770.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.275.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.74.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.3329.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.2271.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.185.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/cats/cat.950.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.829.jpg'
 '/content/drive/MyDrive/고모부_머신러닝/dogncat/training_set/training_set/dogs/dog.396.jpg']
[1 1 0 0 0 0 1 0 1 1]


In [36]:
print(len(train_data))
print(len(train_label_indices))

print(len(valid_data))
print(len(valid_label_indices))


6003
6003
2002
2002


In [43]:
# test dataset 만들기
test_data = cnd_df['path'][cnd_df['type']=='test']
test_data

0       /content/drive/MyDrive/고모부_머신러닝/dognc...
1       /content/drive/MyDrive/고모부_머신러닝/dognc...
2       /content/drive/MyDrive/고모부_머신러닝/dognc...
3       /content/drive/MyDrive/고모부_머신러닝/dognc...
4       /content/drive/MyDrive/고모부_머신러닝/dognc...
                              ...                        
2020    /content/drive/MyDrive/고모부_머신러닝/dognc...
2021    /content/drive/MyDrive/고모부_머신러닝/dognc...
2022    /content/drive/MyDrive/고모부_머신러닝/dognc...
2023    /content/drive/MyDrive/고모부_머신러닝/dognc...
2024    /content/drive/MyDrive/고모부_머신러닝/dognc...
Name: path, Length: 2023, dtype: object

### Custom dataset 만들기

In [64]:
class MyDataset(Dataset):
    def __init__(self, datapath, label=None):
        super(MyDataset, self).__init__()

        self.path = datapath
        self.label = label

    def __len__(self):
        return len(self.path)

    def __getitem__(self,idx):

        image = cv2.cvtColor(cv2.imread(self.path[idx]), cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (244,244))
        image = np.asarray(image, dtype=np.float32).transpose(2,0,1)
        normal_image = (image - np.amin(image)) / (np.amax(image) - np.amin(image))
        
        if self.label is not None:
            label = self.label[idx]

        return normal_image, label


In [65]:
train_dataset = MyDataset(train_data, train_label_indices)
valid_dataset = MyDataset(valid_data, valid_label_indices)
test_dataset = MyDataset(test_data)

In [66]:
type(len(train_dataset))

int

In [167]:
sample_dataset = MyDataset(sample_tr_data, sample_tr_label)

### Loader 만들기

In [67]:
train_loader = DataLoader(train_dataset, batch_size=10, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=10, shuffle=False) # 왜 valid loader에서는 shuffle을 false로 하지?

In [94]:
loader_data = next(iter(train_loader))

In [168]:
sample_loader = DataLoader(sample_dataset, batch_size=2, shuffle=True)

### 모델 만들기

In [176]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 5, kernel_size=3, padding='same')
        self.relu = nn.ReLU()
        self.fc1 = nn.Linear(5*244*244, 2)

    def forward(self, x):
        print('input size :', x.shape)
        print('input max :', torch.amax(x, dim=(1,2,3)))
        print('input min :', torch.amin(x, dim=(1,2,3)))
        print('input l2norm :', torch.linalg.vector_norm(x, dim=(1,2,3)))

        conv = self.conv1(x)
        print('conv size :', conv.shape)
        print('conv max :', torch.amax(conv, dim=(1,2,3)))
        print('conv min :', torch.amin(conv, dim=(1,2,3)))
        print('conv l2norm :', torch.linalg.vector_norm(conv, dim=(1,2,3)))

        conv_out = self.relu(conv)
        print('conv_out size :', conv_out.shape)
        print('conv_out max :', torch.amax(conv_out, dim=(1,2,3)))
        print('conv_out min :', torch.amin(conv_out, dim=(1,2,3)))
        print('conv_out l2norm :', torch.linalg.vector_norm(conv_out, dim=(1,2,3)))

        fc_input = conv_out.view(conv_out.size(0), -1)
        print('fc_input size :', fc_input.shape)
        print('fc_input max :', torch.amax(fc_input, dim=(1)))
        print('fc_input min :', torch.amin(fc_input, dim=(1)))
        print('fc_input l2norm :', torch.linalg.vector_norm(fc_input, dim=(1)))

        fc_logit = self.fc1(fc_input)
        print('fc_logit size :', fc_logit.shape)
        print('fc_logit max :', torch.amax(fc_logit, dim=(1)))
        print('fc_logit min :', torch.amin(fc_logit, dim=(1)))
        print('fc_logit l2norm :', torch.linalg.vector_norm(fc_logit, dim=(1)))
        print()

        return fc_logit

In [177]:
model = CNN().to(device)
model

CNN(
  (conv1): Conv2d(3, 5, kernel_size=(3, 3), stride=(1, 1), padding=same)
  (relu): ReLU()
  (fc1): Linear(in_features=297680, out_features=2, bias=True)
)

### loss function

In [178]:
loss_fn = nn.CrossEntropyLoss().to(device)

### optimzer, lr_scheduler 설정

In [179]:
optimizer = SGD(model.parameters(), lr=0.001)
scheduler = LinearLR(optimizer)

### 모델 training

In [180]:
def train_one_epoch(loader):
    running_loss = 0.
    last_loss = 0.

    for i, data in enumerate(loader):
        image_data, label_data = data
        image_data = image_data.to(device)
        label_data = label_data.to(device)

        optimizer.zero_grad()

        hyphothesis = model(image_data)

        loss = loss_fn(hyphothesis, label_data)

        loss.backward()

        print(f'batch{i+1} loss : {loss}')
        print()
        print(f'gradient max :', torch.amax(torch.tensor([torch.amax(param.grad) for param in model.parameters()])))
        print(f'gradient min :', torch.amin(torch.tensor([torch.amax(param.grad) for param in model.parameters()])))
        print(f'gradient l2norm :', torch.amax(torch.tensor([torch.linalg.vector_norm(param.grad) for param in model.parameters()])))
        print()
        optimizer.step()

        running_loss += loss.item()

        # last_loss = running_loss / len(loader)
        # print(f'Average batch loss :', last_loss)
        # print()
        # last_loss = 0


        if i+1 == len(loader):
            last_loss = running_loss / len(loader)
            print(f'Average batch loss :', last_loss)
            print()
            running_loss = 0

    return last_loss


In [183]:
epoch = 50
for i in range(epoch):
    print(f'========= Epoch{i+1} =========')
    print()
    model.train(True)
    train_one_epoch(sample_loader)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
fc_logit max : tensor([2.3738, 1.5128], device='cuda:0', grad_fn=<AmaxBackward0>)
fc_logit min : tensor([-2.6306, -1.6455], device='cuda:0', grad_fn=<AminBackward0>)
fc_logit l2norm : tensor([3.5433, 2.2352], device='cuda:0', grad_fn=<LinalgVectorNormBackward0>)

batch3 loss : 0.02415481209754944

gradient max : tensor(0.0320)
gradient min : tensor(0.)
gradient l2norm : tensor(4.0051)

input size : torch.Size([2, 3, 244, 244])
input max : tensor([1., 1.], device='cuda:0')
input min : tensor([0., 0.], device='cuda:0')
input l2norm : tensor([261.5967, 229.3072], device='cuda:0')
conv size : torch.Size([2, 5, 244, 244])
conv max : tensor([1.1247, 0.8228], device='cuda:0', grad_fn=<AmaxBackward0>)
conv min : tensor([-0.6742, -0.6172], device='cuda:0', grad_fn=<AminBackward0>)
conv l2norm : tensor([200.4118, 182.4229], device='cuda:0',
       grad_fn=<LinalgVectorNormBackward0>)
conv_out size : torch.Size([2, 5, 244, 244])
conv_out max : tensor([1.1247, 0